## Import library

In [1]:
import numpy as np
import pandas as pd
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#去除停頓詞
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
import spacy  #要先下載好model
from nltk import word_tokenize

#將文字轉換成向量
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.utils import simple_preprocess
from gensim import models
from gensim.models.word2vec import Word2Vec
import gensim

#Create model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import random

#評估模型好壞
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

#if __name__ == "__main__":
#    pass

C:\Users\Administration\anaconda3\envs\Lab_2021\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#關閉煩人的警告視窗!!!!!
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [3]:
df = pd.read_csv('./archive/yelp.csv')
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  10000 non-null  object
 1   date         10000 non-null  object
 2   review_id    10000 non-null  object
 3   stars        10000 non-null  int64 
 4   text         10000 non-null  object
 5   type         10000 non-null  object
 6   user_id      10000 non-null  object
 7   cool         10000 non-null  int64 
 8   useful       10000 non-null  int64 
 9   funny        10000 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 781.4+ KB


## Data Preprocessing

##### a. 讀取csv檔僅保留"text"、"stars"兩個欄位

In [5]:
df1 = df[['stars','text']]
df1.head(20)

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,4,love the gyro plate. Rice is so good and I als...
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,5,General Manager Scott Petello is a good egg!!!...
5,4,"Quiessence is, simply put, beautiful. Full wi..."
6,5,Drop what you're doing and drive here. After I...
7,4,"Luckily, I didn't have to travel far to make m..."
8,4,Definitely come for Happy hour! Prices are ama...
9,5,Nobuo shows his unique talents with everything...


 將stars欄位內值大於等於4的轉成1，其餘轉成0

- 1: positive

- 0: negative

In [6]:
df1.stars[df1.stars < 4] = 0 
df1.stars[df1.stars >= 4] = 1
#一定要先小於4的轉成0, 再將大於等於4的轉成1 不然會全部變成0

In [7]:
#先全部轉成小寫 等等停頓詞處理有差
for i in range(df1.shape[0]):
    df1['text'][i] = df1['text'][i].lower()

In [8]:
df1.head(20)

,stars,text
0,1,my wife took me here on my birthday for breakf...
1,1,i have no idea why some people give bad review...
2,1,love the gyro plate. rice is so good and i als...
3,1,"rosie, dakota, and i love chaparral dog park!!..."
4,1,general manager scott petello is a good egg!!!...
5,1,"quiessence is, simply put, beautiful. full wi..."
6,1,drop what you're doing and drive here. after i...
7,1,"luckily, i didn't have to travel far to make m..."
8,1,definitely come for happy hour! prices are ama...
9,1,nobuo shows his unique talents with everything...


In [9]:
df1.shape

(10000, 2)

將text欄位內的文字利用分割符號切割

In [10]:
"""
df1['split_text'] = df1['text'].str.split(' ')
df1['split_text']
"""

"\ndf1['split_text'] = df1['text'].str.split(' ')\ndf1['split_text']\n"

##### b. 去除停頓詞 stop words 
- 停頓詞 (Stop Words) 的定義上是兩個集合：

1. 這個語言中出現非常頻繁的詞。

2. 文本資料中出現非常頻繁的詞。

網路上有3種做法:

a. spaCy

b. nltk

c. sklearn

In [11]:
#用 Python NLP 中的 spacy 進行stopwords 處理
#先看一下spacy中有哪些停頓詞
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print('spaCy has {} stop words'.format(len(spacy_stopwords)))
print('The first twenty stop words are {}'.format(list(spacy_stopwords)[:20]))

spaCy has 326 stop words
The first twenty stop words are ['a', 'anyhow', 'however', 'into', 'if', 'sixty', 'full', 'would', 'fifteen', 'very', 'please', 'everyone', 'put', 'sometime', 'itself', 'bottom', '‘ll', '‘re', 'in', 're']


In [12]:
"""
1. imdb_df['text'].apply將每個row的text值分別取出
2. apply(lambda x: rmsw_function(x, spacy_stopwords)) 這邊可以拆解成
    - 將imdb_df['text'] 取出，當作lambda function的 x
    - 然後丟進 rmsw_function 得到ruturn的值
    - 然後放到 imdb['spacy_rmsw_text_fancy']之中
"""
#nltk.download('punkt')
#def rmsw_function(text, stopword_list):
#    return ' '.join([word for word in word_tokenize(text) if word not in stopword_list])
#df1['rmsw_text'] = df1['text'].apply(lambda x: rmsw_function(x, spacy_stopwords))
df1['rmsw_text'] =  df1['text'].apply(lambda x:' '.join([word for word in x.split() if word not in spacy_stopwords]))

In [13]:
print(df1['rmsw_text'][0])

wife took birthday breakfast excellent. weather perfect sitting outside overlooking grounds absolute pleasure. waitress excellent food arrived quickly semi-busy saturday morning. looked like place fills pretty quickly earlier better. favor bloody mary. phenomenal simply best i've had. i'm pretty sure use ingredients garden blend fresh order it. amazing. menu looks excellent, white truffle scrambled eggs vegetable skillet tasty delicious. came 2 pieces griddled bread amazing absolutely meal complete. best "toast" i've had. anyway, can't wait back!


In [14]:
print(df1['text'][0])

my wife took me here on my birthday for breakfast and it was excellent.  the weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  our waitress was excellent and our food arrived quickly on the semi-busy saturday morning.  it looked like the place fills up pretty quickly so the earlier you get here the better.

do yourself a favor and get their bloody mary.  it was phenomenal and simply the best i've ever had.  i'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  it was amazing.

while everything on the menu looks excellent, i had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  it came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  it was the best "toast" i've ever had.

anyway, i can't wait to go back!


In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   stars      10000 non-null  int64 
 1   text       10000 non-null  object
 2   rmsw_text  10000 non-null  object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [16]:
df1.shape

(10000, 3)

##### c.文字探勘前處理，將文字轉換成向量，請實作 tf-idf 及 word2vec 並進行比較

##### Tf-idf 

In [17]:
# td-idf
#vectorizer = CountVectorizer(stop_words = spacy_stopwords)
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(df1['rmsw_text'])
weight = tfidf.toarray()
print(weight.shape)
print(weight)
df1['tfidf'] = tfidf

(10000, 29160)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   stars      10000 non-null  int64 
 1   text       10000 non-null  object
 2   rmsw_text  10000 non-null  object
 3   tfidf      10000 non-null  object
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


##### Word2Vec 

In [19]:
tokenized_tweet = df1['rmsw_text'].apply(lambda x: x.split()) # tokenizing 

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            vector_size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
) 


In [20]:
model_w2v.build_vocab(tokenized_tweet, progress_per = 1000)

In [21]:
model_w2v.epochs

5

In [22]:
model_w2v.corpus_count

10000

In [23]:
model_w2v.train(tokenized_tweet, total_examples =model_w2v.corpus_count, epochs = model_w2v.epochs)

(2865961, 3167730)

In [24]:
model_w2v.save("./w2v_model")

In [25]:
model_w2v.wv.most_similar('bad')

[('terrible', 0.6191313862800598),
 ('negative', 0.6006874442100525),
 ('ruined', 0.5913141369819641),
 ('either.', 0.5795227885246277),
 ('worse', 0.5695444345474243),
 ('previous', 0.5682346820831299),
 ('coup,', 0.5675501823425293),
 ('problems', 0.566647469997406),
 ('compelled', 0.5659807920455933),
 ("people's", 0.5637657046318054)]

In [26]:
model_w2v.wv['bad']

array([-2.33778983e-01,  5.12346961e-02,  5.60183585e-01, -3.50196719e-01,
       -3.88479918e-01,  4.57535505e-01,  4.39154170e-02, -7.24750221e-01,
        3.84588450e-01, -2.20163912e-01,  3.51371258e-01, -8.48540187e-01,
       -6.12686992e-01,  4.08979729e-02, -2.24307224e-01,  4.14347053e-01,
        1.18558161e-01,  5.48949838e-01, -2.50739694e-01, -6.02923334e-01,
       -1.47285879e-01,  8.82298723e-02, -7.05669373e-02,  2.44033530e-01,
       -2.96892822e-01,  4.70354199e-01,  1.36866346e-01, -1.86712936e-01,
       -9.75484475e-02,  1.97588757e-01,  3.93760026e-01, -3.95396888e-01,
       -2.75601191e-03,  1.87644303e-01,  4.45496291e-02, -4.52596426e-01,
       -2.70542204e-01, -4.44494814e-01,  1.47487119e-01,  8.38618204e-02,
       -2.23444566e-01, -1.45348862e-01,  1.59412980e-01,  3.61816376e-01,
       -4.15827990e-01,  4.83291686e-01,  4.49441910e-01,  9.52999443e-02,
       -1.40420943e-01, -7.68519998e-01,  4.10884470e-01,  2.03868356e-02,
       -2.37441435e-01,  

In [27]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [28]:
#轉成array的形式等等丟model
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
# wordvec_df.shape
print(len(wordvec_arrays))

10000


In [29]:
df1.head(20)

,stars,text,rmsw_text,tfidf
0,1,my wife took me here on my birthday for breakf...,wife took birthday breakfast excellent. weathe...,"(0, 2307)\t0.08236035721131113\n (0, 28032)..."
1,1,i have no idea why some people give bad review...,"idea people bad reviews place. goes you, every...","(0, 2307)\t0.08236035721131113\n (0, 28032)..."
2,1,love the gyro plate. rice is so good and i als...,love gyro plate. rice good dig candy selection :),"(0, 2307)\t0.08236035721131113\n (0, 28032)..."
3,1,"rosie, dakota, and i love chaparral dog park!!...","rosie, dakota, love chaparral dog park!!! it's...","(0, 2307)\t0.08236035721131113\n (0, 28032)..."
4,1,general manager scott petello is a good egg!!!...,general manager scott petello good egg!!! deta...,"(0, 2307)\t0.08236035721131113\n (0, 28032)..."
5,1,"quiessence is, simply put, beautiful. full wi...","quiessence is, simply put, beautiful. windows ...","(0, 2307)\t0.08236035721131113\n (0, 28032)..."
6,1,drop what you're doing and drive here. after i...,drop you're drive here. ate day more. food goo...,"(0, 2307)\t0.08236035721131113\n (0, 28032)..."
7,1,"luckily, i didn't have to travel far to make m...","luckily, didn't travel far connecting flight. ...","(0, 2307)\t0.08236035721131113\n (0, 28032)..."
8,1,definitely come for happy hour! prices are ama...,"definitely come happy hour! prices amazing, sa...","(0, 2307)\t0.08236035721131113\n (0, 28032)..."
9,1,nobuo shows his unique talents with everything...,nobuo shows unique talents menu. carefully cra...,"(0, 2307)\t0.08236035721131113\n (0, 28032)..."


## Create Model

##### a. 使用 Random forest進行分類 

In [30]:
#先單純跑一次試試看
y=df1['stars']
x=tfidf
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [31]:
rf= RandomForestClassifier(criterion= 'entropy', max_depth= None, max_features= 'sqrt', n_estimators= 500, oob_score = True)
rf.fit(x_train,y_train)
y_pred= rf.predict(x_test)
accuracy_score(y_pred,y_test)

0.7945

In [32]:
#先用套件跑跑看效果
from sklearn.model_selection import cross_val_score
x = tfidf
y = df1['stars']
#rf= RandomForestClassifier(criterion= 'entropy', max_depth= None, max_features= 'sqrt', n_estimators= 100, oob_score = True)
rf = RandomForestClassifier(criterion= 'entropy', max_features= 'sqrt', n_estimators= 500, oob_score = True)
accuracy = cross_val_score(rf, x, y, cv=4, scoring='accuracy')
print(accuracy)
print(accuracy.mean()*100,'%')

[0.7808 0.7944 0.7884 0.7944]
78.95 %


In [39]:
from sklearn.model_selection import cross_val_score
x = tfidf
y = wordvec_df
#rf= RandomForestClassifier(criterion= 'entropy', max_depth= None, max_features= 'sqrt', n_estimators= 100, oob_score = True)
rf = RandomForestClassifier(criterion= 'entropy', max_features= 'sqrt', n_estimators= 500, oob_score = True)
accuracy = cross_val_score(rf, x, y, cv=4, scoring='accuracy')
print(accuracy)
print(accuracy.mean()*100,'%')

[nan nan nan nan]
nan %


將等等要丟入model的資料先分別取出concat起來

In [33]:
a = df1['stars']
b = pd.DataFrame(tfidf.toarray())
df2 = pd.concat([a,b], axis = 1)
df2 = df2.iloc[:10000]

In [34]:
a = df1['stars']
b = wordvec_df
df3 = pd.concat([a,b], axis = 1)
df3 = df3.iloc[:10000]

#### k-fold cross-validation

In [35]:
import random
model= RandomForestClassifier(criterion= 'entropy', max_features= 'sqrt', n_estimators= 100, oob_score = True)
def cross_validation_split(dataset, folds):
        dataset_split = []
        df_copy = dataset
        fold_size = int(df_copy.shape[0] / folds)
        
        # for loop to save each fold
        for i in range(folds):
            fold = []
            # while loop to add elements to the folds
            while len(fold) < fold_size:
                # select a random element
                r = random.randrange(df_copy.shape[0])
                # determine the index of this element 
                index = df_copy.index[r]
                # save the randomly selected line 
                fold.append(df_copy.loc[index].values.tolist())
                # delete the randomly selected line from
                # dataframe not to select again
                df_copy = df_copy.drop(index)
            # save the fold     
            dataset_split.append(np.asarray(fold))
            
        return dataset_split


In [36]:
def K_fold_CV(dataset, f = 4):
    data=cross_validation_split(dataset,f)
    Accuracy=0
    # determine training and test sets 
    for i in range(f):
        r = list(range(f))
        r.pop(i) #i是testing set
        test=data[i]
        
        for j in r :
            if j == r[0]:
                cv = data[j]
            else:    
                cv=np.concatenate((cv,data[j]), axis=0)
                
        # apply RandomForest model
        X = cv[:,1:]
        Y = cv[:,0]
        X_test = test[:,1:]
        y_test = test[:,0]
        model.fit(X,Y)
        y_pred=model.predict(X_test)
        Accuracy=Accuracy+metrics.accuracy_score(y_test, y_pred)
    return Accuracy/f

In [37]:
K_fold_CV(df3) #word2vec

0.7772

In [38]:
K_fold_CV(df2) #tfidf

0.7857999999999999